In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, log_loss
import itertools

In [2]:
# Import data
df0 = pd.read_stata('/Users/nbs/Documents/Georgetown/Semester 5/1 Courses/GBUS 401/1 Project/gbus_401_project/Data_Final/gbus_401_project_master.dta')

# Clean up
df = df0.replace(['False', 'True'], [0, 1])

# Convert year, school_id to dummies for TWFE
fe_catvars = df[['year', 'school_id']]
df = pd.get_dummies(df, prefix=['y', 'sid'], columns=['year', 'school_id'], drop_first=True) # First column is dropped to prevent collinearity
df = df.join(fe_catvars)

# Model 1: Linear Regression

In [3]:
# List of variables to use
varlist = ['admit', 'gpa', 'lsat', 'urm', 'fee_waived', 'non_trad', 'intl', 'year']

# Fixed-effect dummies
for i in list(df.columns):
    if ('sid_' in i) or ('y_' in i):
        varlist.append(i)

# Define dataset for Model 1
df1 = df[varlist]
df1 = df1.dropna(axis='index') # Drop missing

# Define features and outcome
y = df1[['admit']]
X = df1.drop(['admit', 'year'], axis=1)

In [4]:
# Define model
model = LinearRegression(n_jobs=-1)
model.fit(X, y)

# Predict admit
y_hat = np.array([i for i in model.predict(X)])

# Print outputs
print('Coefficients')
[print(a, ':', round(b, 3)) for a, b in zip(model.feature_names_in_[0:6], model.coef_.flatten()[0:6])]
print('')

print('Intercept:', round(model.intercept_.item(), 3), '\n')

print('Goodness of Fit')
print('Cross Entropy:', round(log_loss(y, y_hat), 3)) # issue caused by FE for some reason???
print('R^2', round(model.score(X, y), 3))
print('MSE:', round(mean_squared_error(y, y_hat), 3))

Coefficients
gpa : 0.372
lsat : 0.038
urm : 0.151
fee_waived : 0.047
non_trad : -0.012
intl : -0.055

Intercept: -6.04 

Goodness of Fit
Cross Entropy: 0.436
R^2 0.445
MSE: 0.12


In [21]:
# All possible features
features = ['gpa', 'lsat', 'urm', 'fee_waived', 'non_trad', 'intl']

# Get all combinations of features to test
feature_combos = []
for j in range(1, len(features) + 1):
    feature_combos.append(list(itertools.combinations(features, j)))
feature_combos = list(itertools.chain.from_iterable(feature_combos))
print(type(list(feature_combos[0])))

# Get year and FEs
fes = df1.columns[7:].values.tolist()
print(type(fes))

# Get year
y = ['admit']
print(type(y))

<class 'list'>
<class 'list'>
<class 'list'>


In [24]:
# Cross validation; cite: # https://stackoverflow.com/questions/58069691/how-to-create-a-train-test-split-of-time-series-data-by-year

mr2s = []
mentropies = []

for i in feature_combos:

    # Define data set
    i = list(i)

    flist = []

    for j in i:
        flist.append(i)

    for j in fes:
        flist.append(j)
    
    for j in y:
        flist.append(j)

    print(flist)
    dfi = df1[features_i]

    # Define X and y
    y = dfi[['admit']]
    X = dfi.drop(['admit', 'year'], axis=1)
    
    # Split data set
    year_list = sorted(dfi['year'].unique())
    splits = {'train': [], 'test': []}

    for j, year in enumerate(year_list[:-1]):

        train_year = year_list[:j + 1]
        test_year = [year_list[j + 1]]

        #print('Train:', train_year, 'Test:',test_year)
        
        splits['train'].append(dfi.loc[dfi.year.isin(train_year), :])
        splits['test'].append(dfi.loc[dfi.year.isin(test_year), :])

    # Estimate test statistics

    r2s = []
    entropies = []

    for k in range(len(year_list) - 1):

        X_train, X_test = splits['train'][k][flist], splits['test'][k][flist]
        y_train, y_test = splits['train'][k][['admit']], splits['test'][k][['admit']]

        model.fit(X_train, y_train)

        r2 = model.score(X_test, y_test)
        r2s.append(r2)

        y_test_hat = model.predict(X_test)
        entropy = log_loss(y, y_test_hat) # WHY WON'T THIS WORK?????
        entropies.append(entropy)

    mr2s.append(np.mean(r2s))
    mentropies.append(np.mean(entropies))

[['gpa'], 'year', 'y_2005', 'y_2006', 'y_2007', 'y_2008', 'y_2009', 'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017', 'y_2018', 'y_2019', 'y_2020', 'y_2021', 'y_2022', 'y_2023', 'sid_103600', 'sid_105100', 'sid_108100', 'sid_108300', 'sid_110101', 'sid_110800', 'sid_116453', 'sid_120529', 'sid_121601', 'sid_123456', 'sid_129500', 'sid_130503', 'sid_131273', 'sid_131373', 'sid_131400', 'sid_131500', 'sid_132500', 'sid_132600', 'sid_132800', 'sid_132901', 'sid_134201', 'sid_137000', 'sid_137100', 'sid_140201', 'sid_141706', 'sid_142600', 'sid_143401', 'sid_143711', 'sid_144102', 'sid_144402', 'sid_144503', 'sid_144802', 'sid_146601', 'sid_146800', 'sid_148001', 'sid_148900', 'sid_150900', 'sid_153101', 'sid_153505', 'sid_153601', 'sid_156400', 'sid_157400', 'sid_158015', 'sid_159800', 'sid_161000', 'sid_162600', 'sid_167100', 'sid_169101', 'sid_169800', 'sid_171000', 'sid_173700', 'sid_173900', 'sid_175800', 'sid_177400', 'sid_177500', 'sid_180900', 'sid_181

/Users/nbs/opt/anaconda3/envs/gbus401project/lib/python3.10/site-packages/pandas/core/common.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


TypeError: unhashable type: 'list'

In [ ]:
# Run on test and train data sets



# List of variables to use







In [ ]:
print(list(list(itertools.chain.from_iterable(features))[15]))

['gpa', 'lsat']


# Model 3: Decision Tree

See here for source: https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html#sphx-glr-auto-examples-tree-plot-cost-complexity-pruning-py

In [ ]:
# Split data into testing (25%) and training (75%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run decision tree
clf = DecisionTreeClassifier(random_state=0)
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Train decision tree using effective alphas
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)

# Remove trivial tree with one node
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

# Plot maximum depth vs. alpha
max_depths = [clf.tree_.max_depth for clf in clfs]

fig1 = plt.figure(dpi=150)
plt.scatter(ccp_alphas, max_depths)
plt.plot(ccp_alphas,max_depths, drawstyle="steps-post")
plt.xlabel("Alpha")
plt.ylabel("Maximum Depth")
plt.title("Tree Depth Decreases as Alpha Increases")
plt.show()

# Plot accuracy vs. alpha
train_scores = [clf.score(X_train, y_train) for clf in clfs] # What is the score?
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots(dpi=150)
ax.set_xlabel("Alpha")
ax.set_ylabel("Accuracy")
ax.set_title("Accuracy and Alpha for Training and Testing Data")
ax.plot(ccp_alphas,train_scores,marker="o",label="Train",drawstyle="steps-post")
ax.plot(ccp_alphas,test_scores,marker="o",label="Test",drawstyle="steps-post")
ax.legend()
plt.show()